## (Rass: Third Movement) Second movement: Exponential Defense with Geometric Attack Pattern

In [56]:
print("""This Cell is necessary or else my IDE will start running cells simultaneously which
      then leads to issues with unfinished imports""")

This Cell is necessary or else my IDE will start running cells simultaneously which
      then leads to issues with unfinished imports


In [57]:
import os
import logging
import numpy as np
from datetime import datetime

## New Stuff

In [58]:
# Check if variables exist, otherwise default to False
image_mode = globals().get('image_mode', False)
debug_mode = globals().get('debug_mode', False)
RUN_BASELINE_ONLY = globals().get('RUN_BASELINE_ONLY', False)

# If we are NOT using "extrapolate.ipynb" uncomment any you want to set to true
# however remember to re-comment them out later because otherwise extrapolate will be ignored
#image_mode = True
#debug_mode = True
#RUN_BASELINE_ONLY = True

print(f"Current settings: image_mode={image_mode}, debug_mode={debug_mode}, RUN_BASELINE_ONLY={RUN_BASELINE_ONLY}")

Current settings: image_mode=False, debug_mode=False, RUN_BASELINE_ONLY=False


In [59]:
experiment_name = "experiment_2"

In [60]:
# path construct
log_path = os.path.join(os.getcwd(), str(experiment_name) + '.log')

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

# concat of name/date
logger.info('[1] "' + str(experiment_name) + '.log"')
logger.info('[1] "' + datetime.now().strftime("%a %b %d %H:%M:%S %Y") + '"')

# Set up subgraph logger 
if not RUN_BASELINE_ONLY: 
    subgraph_log_path = os.path.join(os.getcwd(), 'sub_' + str(experiment_name) + '.log')
    
    if os.path.exists(subgraph_log_path):
        os.remove(subgraph_log_path)
        
    subgraph_handler = logging.FileHandler(subgraph_log_path, mode='w')
    subgraph_handler.setFormatter(logging.Formatter('%(message)s'))
    

# Function to switch between loggers
def switch_logger(use_subgraph_logger=False):
    
    # remove handlers
    current_handlers = logger.handlers[:]
    i = 0
    while i < len(current_handlers):
        logger.removeHandler(current_handlers[i]) 
        i = i + 1
        
    if use_subgraph_logger:
        logger.addHandler(subgraph_handler)
        logger.info('[1] "sub_' + str(experiment_name) + '.log"')
        logger.info('[1] "' + datetime.now().strftime("%a %b %d %H:%M:%S %Y") + '"')
    else:
        logger.addHandler(handler)

In [61]:
# Global configuration
# This is the default weight value we will insert any time we we 
# need to insert a weight value for a trivial edge in the graph
# Why 0 ?  Because we have "hardness = exnp(-w)" so we get hardness = 1 for w = 0
# hardness = 1 means that edge is trivial to traverse
DEFAULT_WEIGHT_VALUE = 0  

In [62]:
%run attack_graph_MARA.ipynb

### Create Subgraphs

In [63]:
%run create_subgraphs.ipynb

### Movement Function

In [ ]:
attack_rate_list = [4]  
defense_rate_list = [4]

def random_steps(route, attack_rate=None, defense_rate=None, graph=None): #random_steps
    
    # Geometric distribution
    p = defense_rate / (attack_rate + defense_rate)
    x = np.arange(len(route))
    pmf = p * np.power(1-p, x)
    # Normalize 
    pmf = pmf / pmf.sum()
    return pmf

In [65]:
# %run ctr-core_simple.ipynb
%run exp_ctr-core.ipynb
main()

We start with the baseline graph calculation!
Now we are going to run the subgraph analysis
The subgraph analysis is done!


In [66]:
with open(experiment_name+'.log', 'r') as f:
    print(f.read())

[1] "experiment_2.log"
[1] "Tue Nov 25 12:45:09 2025"



BASELINE RUN: BOTH ATTACKER AND DEFENDER HAVE FULL GRAPH KNOWLEDGE



Debug - Current Attack paths:
  0: [1, 2, 3, 'c(6,9)']
  1: [1, 2, 4, 'c(6,9)']
  2: [1, 2, 5, 7, 8, 'c(6,9)']

+++++++++++++++++++++++++++++

The virtual target nodeid is c(6,9)

attack rate =  1 , defense rate =  4 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
2 0.000000e+00
3 3.249578e-01
4 3.249578e-01
5 0.000000e+00
7 0.000000e+00
8 3.500844e-01

worst case attack strategies per goal:
          1
1 0.3375211
2 0.3375211
3 0.3249578
[1] 0.022

Defender can keep attacker success below: 0.022
Attacker can guarantee success probability of: 0.022

+++++++++++++++++++++++++++++

The virtual target nodeid is c(6,9)

attack rate =  4 , defense rate =  4 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
2 0.000000e+00
3 2.927842e-01
4 2.927842e-01
5 0.000000e+00
7 0.00